In [5]:
# setup
import pandas as pd
from pathlib import Path

In [29]:
## path setup
# choose base for uploading
base_folder = Path().resolve().parent

data_folder = base_folder / "data"
cleaned_folder = data_folder / "clean_data"

# create folder
cleaned_folder.mkdir(parents = True, exist_ok = True)

In [31]:
## upload data
# CPI file path
cpi_path = data_folder / "CPI All Urban Consumers Dairy and Related Products in U.S. City Average.xlsx"

# upload FRED data
cpi_control = pd.read_excel(cpi_path, sheet_name = "Annual")

## milk_df path
milk_df_path = cleaned_folder / "milk_df_clean.xlsx"

# upload milk_df
milk_df = pd.read_excel(milk_df_path)

In [33]:
## data cleaning
# rename columns
cpi_control.columns = ["year", "cpi"] 

# get year column to just be years and not months
cpi_control["year"] = pd.to_datetime(cpi_control["year"]).dt.year

# check to see if it worked
cpi_control.head()

,year,cpi
0,1935,15.325
1,1936,15.983
2,1937,16.567
3,1938,15.667
4,1939,15.058


In [35]:
## setting index for inflation
# identify base year
base_year = 2005
base_cpi = cpi_control.loc[cpi_control["year"] == base_year, "cpi"].iloc[0]

cpi_control["cpi_index"] = cpi_control["cpi"] / base_cpi
cpi_control

cpi_control = cpi_control[cpi_control["year"].isin([2005, 2010, 2016])]

cpi_control

,year,cpi,cpi_index
70,2005,182.383,1.000000
75,2010,199.245,1.092454
81,2016,217.306,1.191482


In [37]:
## merge all datasets
# add CPI into milk_df
milk_df = milk_df.merge(cpi_control[["year", "cpi_index"]],
                        how = "left", left_on = "Year", right_on = "year")

# cleanup
milk_df = milk_df.drop(columns=["year"])

milk_df

,Year,State,Organic,milk_sold,gross_value,feed_costs,marketing_costs,operating_costs,overhead,total_costs,cows_per_farm,output_per_cow,cpi_index
0,2005,Minnesota,1,20.88,23.17,10.11,0.43,14.30,16.31,30.61,59,13058,1.000000
1,2005,New York,1,24.03,26.87,13.08,0.24,16.63,21.37,38.00,53,10956,1.000000
2,2005,Pennsylvania,1,24.23,26.85,13.23,0.31,17.10,14.61,31.71,46,12177,1.000000
3,2005,Vermont,1,24.23,26.87,13.61,0.24,17.62,18.88,36.50,62,11982,1.000000
4,2005,Wisconsin,1,21.19,23.63,13.47,0.22,17.11,14.95,32.07,65,13101,1.000000
5,2010,Minnesota,1,25.17,27.81,14.01,0.25,17.89,21.99,39.88,36,10542,1.092454
6,2010,New York,1,24.29,27.22,11.58,0.39,16.39,18.10,34.49,73,12880,1.092454
7,2010,Pennsylvania,1,28.37,30.92,14.05,0.23,18.81,20.52,39.33,72,12247,1.092454
8,2010,Vermont,1,25.97,28.61,15.88,0.23,21.05,22.45,43.50,60,12145,1.092454
9,2010,Wisconsin,1,27.90,30.89,14.75,0.19,19.85,21.15,41.00,56,12940,1.092454


In [39]:
# export file
inflation_path = cleaned_folder / "milk_df_inflation_adjusted.xlsx"

milk_df.to_excel(inflation_path, index = False)